# ĐỒ ÁN CHUYÊN NGÀNH

## Cài đặt thư viện

In [1]:
!pip install transformers torch bitsandbytes accelerate faiss-cpu ipywidgets fpdf

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.7 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 20.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.8 MB/s eta 0:00:0000:0100:01
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=cce94263ab4bb546324fc5c71708522243721a80b0f3c362ed5fc0f

In [2]:
pip install sentence-transformers chromadb google-generativeai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 87.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

## Import thư viện

In [3]:
import json
import torch
import faiss
import re
import os
import requests
import numpy as np
from datetime import datetime
from typing import List, Dict, Tuple
import ipywidgets as widgets
from collections import defaultdict
from IPython.display import display, clear_output, FileLink
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import google.generativeai as genai

# Đăng nhập Huggingface
login("xxx")

# Cấu hình cache -> tăng tốc độ tải mô hình
os.environ['HF_HOME'] = '/kaggle/working/huggingface_cache'

2025-07-10 13:14:02.275317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752153242.461171      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752153242.524777      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## IOC Extractors

In [4]:
!pip install iocsearcher

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 74.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 102.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.8 MB/s eta 0:00:00
  Created wheel for cashaddress: filename=cashaddress-1.0.6-py3-none-any.whl size=6434 sha256=35528872a7921d5f5e65d86ef6dd89692e7034350dc329dc40caa3ce9226bb0f
  Stored in directory: /root/.cache/pip/wheels/ab/4e/08/a6f380a0bb2a7c4e0654e50e2eb9a3358cc58d159a68

In [6]:
# Search url, hash, domain, ip

from pprint import pprint
import requests
from base64 import urlsafe_b64encode

def search_threatfox(ioc):
    url = "https://threatfox-api.abuse.ch/api/v1/"
    headers = {
        "Auth-Key": "xxx",
        "Content-Type": "application/json"
    }
    data = {
        "query": "search_ioc",
        "search_term": ioc,
        "exact_match": True
    }

    response = requests.post(url, json=data, headers=headers)

    if response.ok:
        res = response.json()
        if res['query_status'] == 'ok':
            result = []
            for i in res['data']:
                result_ioc = {}
                result_ioc['ioc_desc'] = i['ioc_type_desc']
                result_ioc['malware'] = i['malware']
                result_ioc['malware_alias'] = i['malware_alias']
                result_ioc['threat_type'] = i['threat_type']
                result_ioc['threat_type_desc'] = i['threat_type_desc']
            return result
        return None
    else:
        print(f"[!] HTTP {response.status_code} - {response.reason}")
        return None



def search_hash(hash):
    # URL của API
    url = "https://hybrid-analysis.com/api/v2/search/hash"
    
    headers = {
        "accept": "application/json",
        "api-key": "xxx",
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {
        "hash": hash,
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        result = response.json()
        filtered_result = []
        for i in range(len(result)):
          if result[i]['signatures'] != []:
            for j in range(len(result[i]['signatures'])):
                if result[i]['signatures'][j]['threat_level'] != 0 or result[i]['signatures'][j]['attck_id'] != None:
                    sigdict = {}
                    sigdict['name'] = result[i]['signatures'][j]['name']
                    sigdict['description'] = result[i]['signatures'][j]['description']
                    sigdict['threat_level'] = result[i]['signatures'][j]['threat_level']
                    if result[i]['signatures'][j]['attck_id'] != None:
                        sigdict['ttp'] = result[i]['signatures'][j]['attck_id']
                    filtered_result.append(sigdict)
        return filtered_result if filtered_result else None
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

In [7]:
import iocsearcher
from iocsearcher.searcher import Searcher
from pprint import pprint
import json

# with open("/kaggle/input/dynamic-analysis/report-mod.json") as f:
#     raw_dynamic = f.read()
# with open("/kaggle/input/static-analysis/21.json") as f:
#     raw_static = f.read()

def ioc_extractors(raw_dynamic, raw_static):
    print("IOC Extractors")
    raw_dynamic = json.dumps(raw_dynamic)
    raw_static = json.dumps(raw_static)
    ioc = []
    searcher = Searcher()
    # Extract ioc trong dynamic analysis và search Threat intelligence
    dynamic_ioc = searcher.search_data(raw_dynamic)
    for i in dynamic_ioc.copy():
        if i.name not in ["url", "fqdn", "sha1", "ip4", "ip6", "md5", "sha1", "sha256"]:
            dynamic_ioc.remove(i)
    
    for i in dynamic_ioc:
        if i.name == "ip4" or i.name == "ip6" or i.name == "fqdn" or i.name == "url":
            if search_threatfox(i.value) != None:
                tmp = {}
                tmp['type'] = i.name
                tmp['ioc'] = i.value
                tmp['info'] = search_threatfox(i.value)
                ioc.append(tmp)
# Extract ioc trong static analysis và search Threat intelligence
    static_ioc = searcher.search_data(raw_static)
    
    for i in static_ioc.copy():
        if i.name not in ["url", "fqdn", "sha1", "ip4", "ip6", "md5", "sha1", "sha256"]:
            static_ioc.remove(i)
    for i in static_ioc:
        if i.name == "ip4" or i.name == "ip6" or i.name == "fqdn" or i.name == "url":
            if search_threatfox(i.value) != None:
                tmp = {}
                tmp['type'] = i.name
                tmp['ioc'] = i.value
                tmp['info'] = search_threatfox(i.value)
                ioc.append(tmp)


    return ioc


## Cấu hình mô hình BAAI (Embedded model)

In [8]:
# Mô hình BAAI
embedding_model_name = "BAAI/bge-base-en-v1.5"
embedding_model = SentenceTransformer(embedding_model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = embedding_model.to(device)
embedding_model.eval()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

##  Create RAG ChromaDB

In [7]:
import os
import json
import torch
from sentence_transformers import SentenceTransformer
import chromadb

# Load behavior data
with open("/kaggle/input/behaviors-ttp/behaviors_ttp.json", "r", encoding="utf-8") as f:
    behaviors = json.load(f)

# ChromaDB setup
PERSIST_DIR = "/kaggle/working/chroma_db"
client = chromadb.PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(
    name="ttp_behaviors",
    metadata={"description": "Behavior text + TTP mapping"}
)

# Utility: chia batch nhỏ để tránh lỗi giới hạn
def batched(iterable, batch_size):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

# Cấu hình batch encode và batch add
encode_batch_size = 16
insert_batch_size = 1000

# Encode theo batch
ids_all, docs_all, metas_all, embs_all = [], [], [], []

for i in range(0, len(behaviors), encode_batch_size):
    batch = behaviors[i : i + encode_batch_size]
    texts = [b["behavior"] for b in batch]
    embeddings = embedding_model.encode(
        texts,
        batch_size=encode_batch_size,
        convert_to_tensor=True,
        show_progress_bar=False
    ).cpu().tolist()

    for idx, (b, emb) in enumerate(zip(batch, embeddings)):
        ids_all.append(f"behavior-{i+idx}")
        docs_all.append(b["behavior"])
        metas_all.append({"ttp": b["ttp"]})
        embs_all.append(emb)

# Add từng chunk nhỏ vào ChromaDB
for id_chunk, doc_chunk, meta_chunk, emb_chunk in zip(
    batched(ids_all, insert_batch_size),
    batched(docs_all, insert_batch_size),
    batched(metas_all, insert_batch_size),
    batched(embs_all, insert_batch_size)
):
    collection.add(
        ids=id_chunk,
        embeddings=emb_chunk,
        metadatas=meta_chunk,
        documents=doc_chunk
    )


print(f"Inserted {len(ids_all)} behavior embeddings into Chroma.")


AttributeError: 'Client' object has no attribute 'persist'

## Gemini-2.0-flash

In [9]:

import google.generativeai as genai
import os

def get_gemini_response(prompt_text: str) -> str:
    try:

        genai.configure(api_key="xxx")


        generation_config = {
            "temperature": 0.3,
            "top_p": 0.8,
            "top_k": 1,
            "max_output_tokens": 2048,
        }

        # Chọn model để sử dụng
        model = genai.GenerativeModel(
            model_name="gemini-2.0-flash",
            generation_config=generation_config
        )

        response = model.generate_content(prompt_text)
        return response.text
    except Exception as e:
        return f"Đã xảy ra lỗi: {e}"

##  Mistral-7B-Instruct-v0.3

In [9]:

generator_model_name = "mistralai/Mistral-7B-Instruct-v0.3"
hf_token = "xxx"


generator_tokenizer = AutoTokenizer.from_pretrained(
    generator_model_name,
    use_auth_token=hf_token
)
generator_model = AutoModelForCausalLM.from_pretrained(
    generator_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=hf_token
)

generator_tokenizer.pad_token = generator_tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
def mistral_answer(prompt):
    
    inputs = generator_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(generator_model.device)

    with torch.no_grad():
        output = generator_model.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=False,
            repetition_penalty=1.1
        )
    input_len = inputs["input_ids"].shape[1] 
    generated_tokens = output[0][input_len:]
    full_answer = generator_tokenizer.decode(generated_tokens, skip_special_tokens=True)
    if "Response:" in full_answer:
        print("split")
        answer = full_answer.split("Response:", 1)[1].strip()
    else:
        answer = full_answer.strip()
    return answer

##  Meta-Llama-3.1-8B-Instruct

In [10]:

generator_model_name = "meta-llama/Llama-3.1-8B-Instruct"
hf_token = "xxx"


generator_tokenizer = AutoTokenizer.from_pretrained(
    generator_model_name,
    use_auth_token=hf_token
)
generator_model = AutoModelForCausalLM.from_pretrained(
    generator_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=hf_token
)

generator_tokenizer.pad_token = generator_tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [20]:
def meta_llama_answer(prompt):
    
    inputs = generator_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(generator_model.device)

    with torch.no_grad():
        output = generator_model.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=False,
            repetition_penalty=1.1
        )
    input_len = inputs["input_ids"].shape[1] 
    generated_tokens = output[0][input_len:]
    full_answer = generator_tokenizer.decode(generated_tokens, skip_special_tokens=True)

    if "Response:" in full_answer:
        print("split")
        answer = full_answer.split("Response:", 1)[1].strip()
    else:
        answer = full_answer.strip()
    
    return answer

In [ ]:
print(get_gemini_response("Reads information about supported languages \"21.exe\" (Path: \"HKLM\\SYSTEM\\CONTROLSET001\\CONTROL\\NLS\\CUSTOMLOCALE\"; Key: \"EMPTY\")"))

## Xử lý Dynamic Analysis

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

# Hàm trích xuất câu trả lời từ LLM
def extract_balanced_json(text, after_keyword="Return only the output JSON object as described."):
    index = text.find(after_keyword)
    if index == -1:
        print('Key word not found')
        return None
    
    remaining = text[index + len(after_keyword):]
    start = remaining.find('{')
    if start == -1:
        print('Key word json not found')
        return None

    brace_count = 0
    for i in range(start, len(remaining)):
        if remaining[i] == '{':
            brace_count += 1
        elif remaining[i] == '}':
            brace_count -= 1
            if brace_count == 0:
                json_str = remaining[start:i+1]
                try:
                    return json.loads(json_str)
                except json.JSONDecodeError:
                    return json_str.strip()
    return None

def transform_malware_json(input_json: dict, tokenizer, model, device='cuda'):
    
    system_prompt = """You are a JSON-to-Insight transformer for malware dynamic analysis. You only respond in JSON."""
    user_prompt = f"""
    You are a JSON-to-Insight transformer for malware dynamic analysis. You only respond in JSON.
Given a malware analysis report in JSON format, extract and output a new JSON object with the following structure:

{{
    "behavior": <string, taken directly from the "description" field>,
    "evidence": <string or list of strings, semantically summarized from the "marks" field without redundancy>
}}

Instructions:
- Always extract the exact text from the "description" field and place it as the value of the "behavior" key.
- For "evidence":
    - Review all entries in the "marks" array.
    - Summarize each entry into a short semantic description.
    - If multiple entries are semantically similar, include only one.
    - If they are distinct, output them as a list of short meaningful evidence strings.

Now convert this JSON:

{json.dumps(input_json, indent=4)}

Return only the output JSON object as described.

Response: 
"""

    inputs = tokenizer(user_prompt, return_tensors="pt", return_attention_mask=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=1024,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        return extract_balanced_json(output_text)
    except json.JSONDecodeError:
        return {"error": "Model output is not valid JSON", "output": model_response}


In [ ]:

input_data = """

"""
print(type(input_data))
result = transform_malware_json(input_data, generator_tokenizer, generator_model)
print(result)

In [12]:
# with open("/kaggle/input/dynamic-analysis/report-mod.json", "r") as f:
#   data = json.load(f)

def extract_signatures(data):
    signatures = []
    cnt = 1
    for i in data['signatures']:
        tmp = transform_malware_json(i, generator_tokenizer, generator_model)
        if i["ttp"] != {}:
            tmp["ttp"] = list(i["ttp"].keys())
        signatures.append(tmp)
    return signatures

In [13]:
def extract_behavior(data):
    behaviors = []
    # Trích xuất processes behavior
    for j in data["behavior"]["processes"]:
      s = j["process_name"] + " (" + j["process_path"] + ") is excuted by command-line " + j["command_line"]
      behaviors.append(s)
      if j["modules"] != []:  
        for k in j["modules"]:
          s = j["process_name"] + " load module: "
          s += k["basename"] + "(Filepath" + k["filepath"] +")"
          behaviors.append(s)
      if j["calls"] != []:
        for k in j["calls"]:
          s = j["process_name"] + " call: " + k["api"]
          if k["arguments"]:
            s += " with arguments: "
            for key, value in k["arguments"].items():
                if isinstance(value, str) and not value.isascii():
                    continue
                s += f'{key} = {value}, '
          behaviors.append(s)
    # Trích xuất generic behavior
    for x in range(len(data["behavior"]["generic"])):
      if data["behavior"]["generic"][x]["summary"] != {}:
        
        for key, value in data["behavior"]["generic"][x]["summary"].items():
          s = data["behavior"]["generic"][x]["process_name"] + " (Filepath: " + data["behavior"]["generic"][x]["process_path"] + ") "
          if key == "file_created":
            s += "create files: "
            for file in value:
              s += file + ", "
            behaviors.append(s)
          if key == "file_opened":
            s += "open files: "
            for file in value:
              s += file + ", "
            behaviors.append(s)
          if key == "file_read":
            s += "read files: "
            for file in value:
              s += file + ", "
            behaviors.append(s)
          if key == "file_written":
            s += "write files: "
            for file in value:
              s += file + ", "
            behaviors.append(s)
          if key == "file_deleted":
            s += "delete files: "
            for file in value:
              s += file + ", "
            behaviors.append(s)
          if key == "regkey_opened":
            s += "open registry keys: "
            for key in value:
              s += key + ", "
            behaviors.append(s)
          if key == "regkey_read":
            s += "read registry keys: "
            for key in value:
              s += key + ", "
            behaviors.append(s)
          if key == "regkey_written":
            s += "write registry keys: "
            for key in value:
              s += key + ", "
            behaviors.append(s)
          if key == "dll_loaded":
            s += "load dlls: "
            for dll in value:
              s += dll + ", "
            behaviors.append(s)
    if  data["behavior"]["processtree"] != []:
      s = "Process tree of this malware: \n"
    for j in data["behavior"]["processtree"]:
      s += str(j) + "\n"
    behaviors.append(s)
    
    return behaviors

In [14]:
def process_dynamic(dynamic_data):
    print("Process dynamic")
    result = {}
    result["behaviors"] = extract_behavior(dynamic_data)
    result["signatures"] = extract_signatures(dynamic_data)
    return result

##  Xử lý Static Analysis

In [15]:
def process_static(static_data):
    print("Process static")
    result = {}
    file_path = list(static_data.keys())[0]
    for x, y in static_data[file_path]["Plugins"].items():
        result[x] = {}
        for a, b in y.items():
            if a == "plugin_output":
                
                for key, value in b.items():
                    result[x][key] = value
            if a == "summary":
                result[x][a] = b
    return result
                

In [55]:
import json
with open("/kaggle/input/static-analysis/21.json") as f:
    raw_static = json.load(f)
file_path = list(raw_static.keys())[0]
print(process_static(raw_static))

{'strings': {'Contains another PE executable': ['This program cannot be run in DOS mode.'], 'summary': 'Strings found in the binary may indicate undesirable behavior:'}, 'packer': {'info_0': 'Section .data is both writable and executable.', 'summary': 'The PE is possibly packed.'}, 'imports': {'Can access the registry': ['RegCloseKey', 'RegSetValueExA', 'RegOpenKeyExA'], 'Possibly launches other programs': ['CreateProcessA'], 'Functions related to the privilege level': ['OpenProcessToken', 'AdjustTokenPrivileges'], 'summary': 'The PE contains functions mostly used by malware.'}}


## Query RAG

In [16]:
import torch
from sentence_transformers import SentenceTransformer
import chromadb
import shutil

src_dir = "/kaggle/input/chromadb"
dst_dir = "/kaggle/working/chromadb"

shutil.copytree(src_dir, dst_dir, dirs_exist_ok=True)

PERSIST_DIR = "/kaggle/working/chromadb"
client = chromadb.PersistentClient(path=PERSIST_DIR)
collection = client.get_collection(name="ttp_behaviors")

def search_similar_behaviors(behavior_text, top_k=3):
    # Tạo embedding
    embedding = embedding_model.encode([behavior_text], convert_to_tensor=True, show_progress_bar=False).cpu().tolist()[0]

    # Truy vấn từ vector database
    results = collection.query(
        query_embeddings=[embedding],
        n_results=top_k
    )

    # Trích xuất kết quả
    matches = []
    for doc, meta, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        if score <= 0.4:
            matches.append({
                "behavior": doc,
                "ttp": meta["ttp"]
                #"score": score
            })

    return matches

In [27]:
print(search_similar_behaviors("Reads information about supported languages \"21.exe\" (Path: \"HKLM\\SYSTEM\\CONTROLSET001\\CONTROL\\NLS\\CUSTOMLOCALE\"; Key: \"EMPTY\")': ['Overall entropy of this PE file is high']}"))

[{'behavior': 'Reads information about supported languages "3fc83a5369f55298d3513048f8a0ca78184b5cad8abf7d78f6728d62c19dcc70.exe" (Path: "HKLM\\SYSTEM\\CONTROLSET001\\CONTROL\\NLS\\CUSTOMLOCALE"; Key: "EMPTY")', 'ttp': 'T1082', 'score': 0.27553480863571167}]


## Web

In [17]:

import json
import re
import copy
from collections import Counter
import pandas as pd

def extract_first_json_array(text):
    matches = re.finditer(r'\[\s*{.*?}\s*]', text, re.DOTALL)
    for match in matches:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            continue
    return None

def split_into_chunks(data_list, chunk_size):
    return [data_list[i:i + chunk_size] for i in range(0, len(data_list), chunk_size)]


def infer_apt(observed_ttps):
    with open("/kaggle/input/ttp-map-apt/ttp_map_group.json", "r") as f:
        ttp_to_groups = json.load(f)
    with open("/kaggle/input/group-map-name/group_map_name.json", "r") as f:
        group_map_name = json.load(f)
    count = Counter()
    for ttp in observed_ttps:
        for group in ttp_to_groups.get(ttp, []):
            count[group] += 1
    if not count:
        return pd.DataFrame(columns=["Rank", "Group", "Score"])

    sorted_groups = sorted(count.items(), key=lambda x: (-x[1], x[0]))
    ranked_apt = []
    rank = 1
    prev_score = None
    for i, (group, score) in enumerate(sorted_groups, 1):
        if score != prev_score:
            rank = i
            prev_score = score
        ranked_apt.append({"Rank": rank, "Group": group_map_name[group], "Score": score})
    return pd.DataFrame(ranked_apt)


def generate_answer(static_json_file, dynamic_json_file):
    messages = []
    if static_json_file is None:
        return "Static Analysis file is missing.", None, None
    if dynamic_json_file is None:
        return "Dynamic Analysis file is missing.", None, None
    try:
        with open(static_json_file, "r") as f:
            static_data = json.load(f)

    except Exception as e:
        messages.append(f"Error loading Static JSON: {e}")
        return "\n".join(messages), None, None

    try:
        with open(dynamic_json_file, "r") as f:
            dynamic_data = json.load(f)

    except Exception as e:
        messages.append(f"Error loading Dynamic JSON: {e}")
        return "\n".join(messages), None, None

    ioc = ioc_extractors(dynamic_data, static_data)
    processed_static = process_static(static_data)
    processed_dynamic = process_dynamic(dynamic_data)
    tmp_processed_dynamic = copy.deepcopy(processed_dynamic)
    final_result = {}
    #print("Processed dynamic before filter: ", len(processed_dynamic["behaviors"]), len(processed_dynamic["signatures"]))

    for key, value in tmp_processed_dynamic.items():
        if key == "behaviors":
            for b in range(len(value)):
                tmp = search_similar_behaviors(value[b])
                if tmp == []:
                    processed_dynamic["behaviors"].remove(value[b])
        if key == "signatures":
            for b in range(len(value)):
                if value[b] == None: continue
                if "ttp" not in list(value[b].keys()):
                    tmp = []
                    if search_similar_behaviors(value[b]["behavior"]) != []:
                        tmp.append(search_similar_behaviors(value[b]["behavior"]))
                    for j in value[b]["evidence"]:
                        tmp_str = value[b]["behavior"] + j
                        if search_similar_behaviors(tmp_str) != []:
                            tmp.append(search_similar_behaviors(tmp_str))
                    if tmp == []:
                        processed_dynamic["signatures"].remove(value[b])

    #print("Processed dynamic after filter: ", len(processed_dynamic["behaviors"]), len(processed_dynamic["signatures"]))
    chunks = split_into_chunks(processed_dynamic["behaviors"], chunk_size=10)
    #print(len(chunks))
    for num, chunk in enumerate(chunks):
        rag_result = []
        seen = set()
        for b in range(len(chunk)):
                tmp = search_similar_behaviors(chunk[b])
                for i in tmp:
                    if i["behavior"] not in seen:
                        seen.add(i["behavior"])
                        rag_result.append(i)
        prompt = f"""You are a cybersecurity analyst specialized in malware behavior analysis.
                Your task is to extract only valid MITRE ATT&CK Tactics, Techniques, and Procedures (TTPs) used by a suspicious file, based on multiple sources of analysis.
                Use the following input sources:
                static_analysis: Static characteristics such as suspicious strings, structurally detect packer presence, suspicious imports.
                dynamic_analysis: Runtime signatures behavior, some behaviors is assigned TTP by sandbox.
                iocs: Indicators of Compromise such as IPs, domains, urls and their information from Threat Intelligence.
                behavior_ttp_mapping: A list of pre-annotated behaviors with candidate TTPs from RAG. You can use this for reference only, but do not copy or cite evidence from this source.
                
                Reasoning Process (Chain-of-thought):
                For each observed behavior or pattern:
                - Determine whether it corresponds to a known MITRE ATT&CK technique.
                - Justify the mapping by explaining the intent or effect of the behavior.
                - Extract the correct MITRE ATT&CK technique ID and name.
                - Identify exact evidence strings from either static_analysis, dynamic_analysis, or iocs only.
                - Do not use any evidence from behavior_ttp_mapping for citations.
                - Only include mappings that are confident and valid. Avoid weak or speculative mappings.

                Response only following output format (JSON) and do not add anything else:
                Output Format (JSON array):
                    
                "technique_id": "Txxxx",
                "evidence": [
                      "Exact evidence string 1 from static/dynamic/IoC",
                      "Exact evidence string 2 from static/dynamic/IoC"
                ],
                "reasoning": "Explanation of why this evidence supports the mapping to the technique."

                Note:
                – Merge multiple supporting behaviors into a single TTP mapping when appropriate  
                – Avoid duplicate TTP entries  
                – Only use official MITRE ATT&CK techniques
                
                Begin your analysis using the provided inputs:
                static_analysis:
                {processed_static}
                dynamic_analysis:
                {chunk}
                iocs:
                {ioc}
                behavior_ttp_mapping:
                {rag_result}
                
                Response:
                        """
        ans = extract_first_json_array(get_gemini_response(prompt))
        #print(get_gemini_response(prompt))
        #print(f"Response chunk {num}: \n{ans}")
        # Thêm vào mảng kết quả
        if ans != None:
            for ans_i in ans:
                if "technique_id" not in ans_i.keys():
                    print(ans_i)
                    continue
                if ans_i['technique_id'] not in final_result.keys():
                    final_result[ans_i['technique_id']] = []
                    tmp = {}
                    tmp["evidence"] = ans_i['evidence']
                    tmp["reasoning"] = ans_i['reasoning']
                    final_result[ans_i['technique_id']].append(tmp)
                else:
                    tmp = {}
                    tmp["evidence"] = ans_i['evidence']
                    tmp["reasoning"] = ans_i['reasoning']
                    final_result[ans_i['technique_id']].append(tmp)
    #print("Processed dynamic after filter: ", len(processed_dynamic["behaviors"]), len(processed_dynamic["signatures"]))
    rag_result = []
    seen1 = set()
    for items in processed_dynamic["signatures"]:
        if items == None: continue
        if "ttp" not in list(items.keys()):
            tmp = []
            if search_similar_behaviors(items["behavior"]) != []:
                tmp.append(search_similar_behaviors(items["behavior"]))
            for j in items["evidence"]:
                tmp_str = items["behavior"] + j
                if search_similar_behaviors(tmp_str) != []:
                    tmp.append(search_similar_behaviors(tmp_str))
            for i in tmp:
                for k in i:
                    if k["behavior"] not in seen1:
                        seen1.add(k["behavior"])
                        rag_result.append(k)
    prompt = f"""You are a cybersecurity analyst specialized in malware behavior analysis.
                Your task is to extract only valid MITRE ATT&CK Tactics, Techniques, and Procedures (TTPs) used by a suspicious file, based on multiple sources of analysis.
                Use the following input sources:
                static_analysis: Static characteristics such as suspicious strings, structurally detect packer presence, suspicious imports.
                dynamic_analysis: Runtime signatures behavior, some behaviors is assigned TTP by sandbox.
                iocs: Indicators of Compromise such as IPs, domains, urls and their information from Threat Intelligence.
                behavior_ttp_mapping: A list of pre-annotated behaviors with candidate TTPs from RAG. You can use this for reference only, but do not copy or cite evidence from this source.
                
                Reasoning Process (Chain-of-thought):
                For each observed behavior or pattern:
                - Determine whether it corresponds to a known MITRE ATT&CK technique.
                - Justify the mapping by explaining the intent or effect of the behavior.
                - Extract the correct MITRE ATT&CK technique ID and name.
                - Identify exact evidence strings from either static_analysis, dynamic_analysis, or iocs only.
                - Do not use any evidence from behavior_ttp_mapping for citations.
                - Only include mappings that are confident and valid. Avoid weak or speculative mappings.

                Response only following output format (JSON) and do not add anything else:
                Output Format (JSON array):
                    
                "technique_id": "Txxxx",
                "evidence": [
                      "Exact evidence string 1 from static/dynamic/IoC",
                      "Exact evidence string 2 from static/dynamic/IoC"
                ],
                "reasoning": "Explanation of why this evidence supports the mapping to the technique."

                Note:
                – Merge multiple supporting behaviors into a single TTP mapping when appropriate  
                – Avoid duplicate TTP entries  
                – Only use official MITRE ATT&CK techniques
                
                Begin your analysis using the provided inputs:
                static_analysis:
                {processed_static}
                dynamic_analysis:
                {processed_dynamic["signatures"]}
                iocs:
                {ioc}
                behavior_ttp_mapping:
                {rag_result}
                
                Response:
                        """
    ans = extract_first_json_array(get_gemini_response(prompt))
    #print(f"Response signatures: \n{ans}")
     # Thêm vào mảng kết quả
    if ans != None:
        for ans_i in ans:
            if "technique_id" not in ans_i.keys():
                print(ans_i)
                continue
            if ans_i['technique_id'] not in final_result.keys():
                final_result[ans_i['technique_id']] = []
                tmp = {}
                tmp["evidence"] = ans_i['evidence']
                tmp["reasoning"] = ans_i['reasoning']
                final_result[ans_i['technique_id']].append(tmp)
            else:
                tmp = {}
                tmp["evidence"] = ans_i['evidence']
                tmp["reasoning"] = ans_i['reasoning']
                final_result[ans_i['technique_id']].append(tmp)

    messages.append(f"Extracted {len(final_result)} techniques.")
    apt_df = infer_apt(list(final_result.keys()))
    return "\n".join(messages), final_result, apt_df
    

In [19]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## DACN")

    with gr.Row():
        static_input = gr.File(label="Upload Static Analysis (JSON)")
        dynamic_input = gr.File(label="Upload Dynamic Analysis (JSON)")

    generate_btn = gr.Button("Generate Answer")

    output_log = gr.Textbox(label="Log / Status")
    output_json = gr.JSON(label="Extracted TTPs")
    output_df = gr.Dataframe(label="APT Groups List")


    generate_btn.click(
        fn=generate_answer,
        inputs=[static_input, dynamic_input],
        outputs=[output_log, output_json, output_df]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://0ee5454b06a4b130d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


IOC Extractors
Process static
Process dynamic


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


##  Đánh giá tổng

In [20]:
import requests
def search_hashv2(hash):
    # URL của API
    url = "https://hybrid-analysis.com/api/v2/search/hash"
    
    headers = {
        "accept": "application/json",
        "api-key": "s64d1p2o088bf64fcgpxlhu9fe9ef89a1mugw4bd2acb3098noaetygs2fb9cc9d", 
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {
        "hash": hash, 
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        result = response.json()
        filtered_result = {}
        for i in range(len(result)):
          if result[i]['signatures'] != []:
            for j in range(len(result[i]['signatures'])):
                if result[i]['signatures'][j]['attck_id'] != None:
                    if result[i]['signatures'][j]['attck_id'] not in list(filtered_result.keys()):
                        filtered_result[result[i]['signatures'][j]['attck_id']] = []
                    for k in result[i]['signatures'][j]['description'].split("\n"):
                        filtered_result[result[i]['signatures'][j]['attck_id']].append(k)
        return filtered_result if filtered_result != [] else None
    else:
        print(f"hash - Request failed with status code {response.status_code}")
        return None

In [19]:
import copy

import re
import json
# Hàm trích xuất câu trả lời từ LLM
def extract_first_json_array(text):
    # Tìm tất cả các đoạn giống mảng JSON
    matches = re.finditer(r'\[\s*{.*?}\s*]', text, re.DOTALL)
    for match in matches:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            continue
    return None

from collections import Counter

def infer_apt(observed_ttps):
    with open("/kaggle/input/ttp-map-apt/ttp_map_group.json", "r") as f:
        ttp_to_groups = json.load(f)
    count = Counter()
    for ttp in observed_ttps:
        for group in ttp_to_groups.get(ttp, []):
            count[group] += 1

    if not count:
        return []

    # Sắp xếp giảm dần theo tần suất
    sorted_groups = sorted(count.items(), key=lambda x: (-x[1], x[0]))
    
    # Tạo list kết quả
    ranked_apt = []
    
    # Biến lưu thông tin rank
    rank = 1
    prev_score = None
    
    for i, (group, score) in enumerate(sorted_groups, 1):
        if score != prev_score:
            rank = i
            prev_score = score
  
        ranked_apt.append((group, score, rank))
    
    return ranked_apt

# Biến trung gian
cti_texts, index = [], None
static_texts, dynamic_texts, malware_texts, report_data = [], [], [], []

def split_into_chunks(data_list, chunk_size):
    return [data_list[i:i+chunk_size] for i in range(0, len(data_list), chunk_size)]

# Xử lý báo cáo JSON
def on_process_report_clicked(static_path, dynamic_path):
    global static_texts
    global dynamic_texts
    with open(static_path, 'r', encoding='utf-8') as f:
        static_texts = json.load(f)
    with open(dynamic_path, 'r', encoding='utf-8') as f:
        dynamic_texts = json.load(f)



def on_generate_clicked():    
    final_result = {}
    ioc = ioc_extractors(dynamic_texts, static_texts)
    processed_static = process_static(static_texts)
    processed_dynamic = process_dynamic(dynamic_texts)
    tmp_processed_dynamic = copy.deepcopy(processed_dynamic)
    
    print("Processed dynamic before filter: ", len(processed_dynamic["behaviors"]), len(processed_dynamic["signatures"]))
    # rag_result = []
    # seen = set()
    for key, value in tmp_processed_dynamic.items():
        if key == "behaviors":
            for b in range(len(value)):
                tmp = search_similar_behaviors(value[b])
                if tmp == []:
                    processed_dynamic["behaviors"].remove(value[b])
                # else:
                #     for i in tmp:
                #         if i["behavior"] not in seen:
                #             seen.add(i["behavior"])
                #             rag_result.append(i)
        if key == "signatures":
            for b in range(len(value)):
                if value[b] == None: continue
                if "ttp" not in list(value[b].keys()):
                    tmp = []
                    if search_similar_behaviors(value[b]["behavior"]) != []:
                        tmp.append(search_similar_behaviors(value[b]["behavior"]))
                    for j in value[b]["evidence"]:
                        tmp_str = value[b]["behavior"] + j
                        if search_similar_behaviors(tmp_str) != []:
                            tmp.append(search_similar_behaviors(tmp_str))
                    if tmp == []:
                        processed_dynamic["signatures"].remove(value[b])
                    # else:
                    #     for item in tmp:
                    #         for k in item:
                    #             if k["behavior"] not in seen:
                    #                 seen.add(k["behavior"])
                    #                 rag_result.append(k)
    print("Processed dynamic after filter: ", len(processed_dynamic["behaviors"]), len(processed_dynamic["signatures"]))
    chunks = split_into_chunks(processed_dynamic["behaviors"], chunk_size=10)
    print(len(chunks))
    # Bỏ những mẫu có số lượng chunk lớn hơn 15 vì giới hạn api
    if len(chunks) > 15: return None
    for num, chunk in enumerate(chunks):
        rag_result = []
        seen = set()
        for b in range(len(chunk)):
                tmp = search_similar_behaviors(chunk[b])
                for i in tmp:
                    if i["behavior"] not in seen:
                        seen.add(i["behavior"])
                        rag_result.append(i)
        prompt = f"""You are a cybersecurity analyst specialized in malware behavior analysis.
                Your task is to extract only valid MITRE ATT&CK Tactics, Techniques, and Procedures (TTPs) used by a suspicious file, based on multiple sources of analysis.
                Use the following input sources:
                static_analysis: Static characteristics such as suspicious strings, structurally detect packer presence, suspicious imports.
                dynamic_analysis: Runtime signatures behavior, some behaviors is assigned TTP by sandbox.
                iocs: Indicators of Compromise such as IPs, domains, urls and their information from Threat Intelligence.
                behavior_ttp_mapping: A list of pre-annotated behaviors with candidate TTPs from RAG. You can use this for reference only, but do not copy or cite evidence from this source.
                
                Reasoning Process (Chain-of-thought):
                For each observed behavior or pattern:
                - Determine whether it corresponds to a known MITRE ATT&CK technique.
                - Justify the mapping by explaining the intent or effect of the behavior.
                - Extract the correct MITRE ATT&CK technique ID and name.
                - Identify exact evidence strings from either static_analysis, dynamic_analysis, or iocs only.
                - Do not use any evidence from behavior_ttp_mapping for citations.
                - Only include mappings that are confident and valid. Avoid weak or speculative mappings.
                
                Response only following output format (JSON) and do not add anything else:
                Output Format (JSON array):
                    
                "technique_id": "Txxxx",
                "evidence": [
                      "Exact evidence string 1 from static/dynamic/IoC",
                      "Exact evidence string 2 from static/dynamic/IoC"
                ],
                "reasoning": "Explanation of why this evidence supports the mapping to the technique."
                
                Note:
                – Merge multiple supporting behaviors into a single TTP mapping when appropriate  
                – Avoid duplicate TTP entries  
                – Only use official MITRE ATT&CK techniques
                
                Begin your analysis using the provided inputs:
                static_analysis: 
                {processed_static}
                dynamic_analysis:
                {chunk}
                iocs:
                {ioc}
                behavior_ttp_mapping:
                {rag_result}
                
                Response:
                        """
        ans = extract_first_json_array(get_gemini_response(prompt))
        #print(f"Response chunk {num}: \n{get_gemini_response(prompt)}")
        if ans != None:
            for ans_i in ans:
                if "technique_id" not in ans_i.keys():
                    print(ans_i)
                    continue
                if ans_i['technique_id'] not in final_result.keys():
                    final_result[ans_i['technique_id']] = ans_i['evidence']
                else:
                    for e in ans_i['evidence']:
                        final_result[ans_i['technique_id']].append(e)
    #print("Processed dynamic after filter: ", len(processed_dynamic["behaviors"]), len(processed_dynamic["signatures"]))
    rag_result = []
    seen1 = set()
    for items in processed_dynamic["signatures"]:
        if items == None: continue
        if "ttp" not in list(items.keys()):
            tmp = []
            if search_similar_behaviors(items["behavior"]) != []:
                tmp.append(search_similar_behaviors(items["behavior"]))
            for j in items["evidence"]:
                tmp_str = items["behavior"] + j
                if search_similar_behaviors(tmp_str) != []:
                    tmp.append(search_similar_behaviors(tmp_str))
            for i in tmp:
                for k in i:
                    if k["behavior"] not in seen1:
                        seen1.add(k["behavior"])
                        rag_result.append(k)
    prompt = f"""You are a cybersecurity analyst specialized in malware behavior analysis.
                Your task is to extract only valid MITRE ATT&CK Tactics, Techniques, and Procedures (TTPs) used by a suspicious file, based on multiple sources of analysis.
                Use the following input sources:
                static_analysis: Static characteristics such as suspicious strings, structurally detect packer presence, suspicious imports.
                dynamic_analysis: Runtime signatures behavior, some behaviors is assigned TTP by sandbox.
                iocs: Indicators of Compromise such as IPs, domains, urls and their information from Threat Intelligence.
                behavior_ttp_mapping: A list of pre-annotated behaviors with candidate TTPs from RAG. You can use this for reference only, but do not copy or cite evidence from this source.
                
                Reasoning Process (Chain-of-thought):
                For each observed behavior or pattern:
                - Determine whether it corresponds to a known MITRE ATT&CK technique.
                - Justify the mapping by explaining the intent or effect of the behavior.
                - Extract the correct MITRE ATT&CK technique ID and name.
                - Identify exact evidence strings from either static_analysis, dynamic_analysis, or iocs only.
                - Do not use any evidence from behavior_ttp_mapping for citations.
                - Only include mappings that are confident and valid. Avoid weak or speculative mappings.

                Response only following output format (JSON) and do not add anything else:
                Output Format (JSON array):
                    
                "technique_id": "Txxxx",
                "evidence": [
                      "Exact evidence string 1 from static/dynamic/IoC",
                      "Exact evidence string 2 from static/dynamic/IoC"
                ],
                "reasoning": "Explanation of why this evidence supports the mapping to the technique."
                
                Note:
                – Merge multiple supporting behaviors into a single TTP mapping when appropriate  
                – Avoid duplicate TTP entries  
                – Only use official MITRE ATT&CK techniques
                
                Begin your analysis using the provided inputs:
                static_analysis:
                {processed_static}
                dynamic_analysis:
                {processed_dynamic["signatures"]}
                iocs:
                {ioc}
                behavior_ttp_mapping:
                {rag_result}

                Response:
                        """
    ans = extract_first_json_array(get_gemini_response(prompt))
    #print(f"Response signatures: \n{get_gemini_response(prompt)}")
    if ans != None:
        for ans_i in ans:
            if ans_i['technique_id'] not in final_result.keys():
                final_result[ans_i['technique_id']] = ans_i['evidence']
            else:
                for e in ans_i['evidence']:
                    final_result[ans_i['technique_id']].append(e)
    return final_result



In [21]:
def calc_rr(rank_list, ground_truth):
    rank = None
    for item in rank_list:
        group_id, score, r = item
        if group_id == ground_truth:
            rank = r
            print(group_id, score, r)
            break
    
    if rank is not None:
        return 1 / rank
    else:
        return 0


In [ ]:
import os
import time
import json

REPORT_DIR = "/kaggle/input/dataset-apt-malware/Winnti/"
with open('/kaggle/input/eval-file/eval_file.json', 'r') as f:
    eval_file = json.load(f)
cnt = 0
CR = 0
cntv2 = 0

for file in os.listdir(REPORT_DIR):
    if cnt == 5: break
    if file in eval_file: continue
    print(file)
    eval_file.append(file)
    with open('/kaggle/working/eval_file.json', 'w') as f:
        json.dump(eval_file, f, indent=4)
   
    cnt += 1
    report = REPORT_DIR + file
    ground = search_hashv2(file)
    while ground == None:
        ground = search_hashv2(file)
        cntv2 += 1
        if cntv2 == 10:
            break
    cntv2 = 0
    if ground == None:
        cnt -= 1
        continue
    static_path = report + "/" + file + ".json"
    dynamic_path = report + "/" + "report.json"
    start_time = time.time()
    on_process_report_clicked(static_path, dynamic_path)
    final_result = on_generate_clicked()
    if final_result == None: 
        cnt-=1
        end_time = time.time()
        continue
    #print(final_result)
    file_result = "/kaggle/working/3-" + file + ".json"
    with open(file_result, 'w') as f:
        json.dump(final_result, f, indent=4)
    set_g = set(list(final_result.keys()))
    set_r = set(list(ground.keys())) 
    print(len(set_g & set_r))
    print(len(set_g))
    print(len(set_r))
    CR += len(set_g & set_r) / len(set_r)
    print(f"{file} - CR_sum = {CR:.4f} - CR_cur = {(len(set_g & set_r) / len(set_r)):.4f}")
    print("Reciprocal Rank (RR):", calc_rr(infer_apt(list(final_result.keys())), "G0016"), "Ground Truth:", os.path.basename(os.path.normpath(REPORT_DIR)))
    end_time = time.time()

    elapsed_time = end_time - start_time
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = elapsed_time % 60
    
    print(f"Run time: {hours} h {minutes} m {seconds:.2f} s - {elapsed_time:.2f}s")



## Đánh giá chỉ LLM

In [21]:
import requests
def search_hashv2(hash):
    # URL của API
    url = "https://hybrid-analysis.com/api/v2/search/hash"
    
    headers = {
        "accept": "application/json",
        "api-key": "s64d1p2o088bf64fcgpxlhu9fe9ef89a1mugw4bd2acb3098noaetygs2fb9cc9d", 
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {
        "hash": hash,
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        result = response.json()
        filtered_result = {}
        for i in range(len(result)):
          if result[i]['signatures'] != []:
            for j in range(len(result[i]['signatures'])):
                if result[i]['signatures'][j]['attck_id'] != None:
                    if result[i]['signatures'][j]['attck_id'] not in list(filtered_result.keys()):
                        filtered_result[result[i]['signatures'][j]['attck_id']] = []
                    for k in result[i]['signatures'][j]['description'].split("\n"):
                        filtered_result[result[i]['signatures'][j]['attck_id']].append(k)
        return filtered_result if filtered_result != [] else None
    else:
        print(f"hash - Request failed with status code {response.status_code}")
        return None

In [16]:
import copy

import re
import json

# Hàm trích xuất câu trả lời từ LLM
def extract_first_json_array(text):
    # Tìm tất cả các đoạn giống mảng JSON
    matches = re.finditer(r'\[\s*{.*?}\s*]', text, re.DOTALL)
    for match in matches:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            continue
    return None

from collections import Counter

def infer_apt(observed_ttps):
    with open("/kaggle/input/ttp-map-apt/ttp_map_group.json", "r") as f:
        ttp_to_groups = json.load(f)
    count = Counter()
    for ttp in observed_ttps:
        for group in ttp_to_groups.get(ttp, []):
            count[group] += 1

    if not count:
        return []

    # Sắp xếp giảm dần theo tần suất
    sorted_groups = sorted(count.items(), key=lambda x: (-x[1], x[0]))
    
    # Tạo list kết quả
    ranked_apt = []
    
    # Biến lưu thông tin rank
    rank = 1
    prev_score = None
    
    for i, (group, score) in enumerate(sorted_groups, 1):
        if score != prev_score:
            rank = i
            prev_score = score
        ranked_apt.append((group, score, rank))

    return ranked_apt

# Biến trung gian
cti_texts, index = [], None
static_texts, dynamic_texts, malware_texts, report_data = [], [], [], []

def split_into_chunks(data_list, chunk_size):
    return [data_list[i:i+chunk_size] for i in range(0, len(data_list), chunk_size)]

# Xử lý báo cáo JSON
def on_process_report_clicked(static_path, dynamic_path):
    global static_texts
    global dynamic_texts
    with open(static_path, 'r', encoding='utf-8') as f:
        static_texts = json.load(f)
    with open(dynamic_path, 'r', encoding='utf-8') as f:
        dynamic_texts = json.load(f)



def on_generate_clicked():    
    final_result = {}
    ioc = ioc_extractors(dynamic_texts, static_texts)
    processed_static = process_static(static_texts)
    processed_dynamic = process_dynamic(dynamic_texts)
    tmp_processed_dynamic = copy.deepcopy(processed_dynamic)
    
    chunks = split_into_chunks(processed_dynamic["behaviors"], chunk_size=14)
    print(len(chunks))
    if len(chunks) > 15: return None
    for num, chunk in enumerate(chunks):
        rag_result = []
        seen = set()

        prompt = f"""You are a cybersecurity analyst specialized in malware behavior analysis.
                Your task is to extract only valid MITRE ATT&CK Tactics, Techniques, and Procedures (TTPs) used by a suspicious file, based on multiple sources of analysis.
                Use the following input sources:
                static_analysis: Static characteristics such as suspicious strings, structurally detect packer presence, suspicious imports.
                dynamic_analysis: Runtime signatures behavior, some behaviors is assigned TTP by sandbox.
                
                Reasoning Process (Chain-of-thought):
                For each observed behavior or pattern:
                - Determine whether it corresponds to a known MITRE ATT&CK technique.
                - Justify the mapping by explaining the intent or effect of the behavior.
                - Extract the correct MITRE ATT&CK technique ID and name.
                - Identify exact evidence strings from either static_analysis, dynamic_analysis, or iocs only.
                - Do not use any evidence from behavior_ttp_mapping for citations.
                - Only include mappings that are confident and valid. Avoid weak or speculative mappings.
                
                Response only following output format (JSON) and do not add anything else:
                Output Format (JSON array):
                    
                "technique_id": "Txxxx",
                "evidence": [
                      "Exact evidence string 1 from static/dynamic/IoC",
                      "Exact evidence string 2 from static/dynamic/IoC"
                ],
                "reasoning": "Explanation of why this evidence supports the mapping to the technique."

                Note:
                – Merge multiple supporting behaviors into a single TTP mapping when appropriate  
                – Avoid duplicate TTP entries  
                – Only use official MITRE ATT&CK techniques
                
                Begin your analysis using the provided inputs:
                static_analysis:
                {processed_static}
                
                dynamic_analysis:
                {chunk}
                
                Response
                        """
        ans = extract_first_json_array(get_gemini_response(prompt))
        if ans != None:
            for ans_i in ans:
                if "technique_id" not in ans_i.keys():
                    print(ans_i)
                    continue
                if ans_i['technique_id'] not in final_result.keys():
                    final_result[ans_i['technique_id']] = ans_i['evidence']
                else:
                    for e in ans_i['evidence']:
                        final_result[ans_i['technique_id']].append(e)

    prompt = f"""You are a cybersecurity analyst specialized in malware behavior analysis and MITRE ATT&CK mapping.
                Your task is to extract only valid MITRE ATT&CK Tactics, Techniques, and Procedures (TTPs) used by a suspicious file, based on multiple sources of analysis.
                Use the following input sources:
                static_analysis: Static characteristics such as suspicious strings, structurally detect packer presence, suspicious imports.
                dynamic_analysis: Runtime signatures behavior, some behaviors is assigned TTP by sandbox.

                Reasoning Process (Chain-of-thought):
                For each observed behavior or pattern:
                - Determine whether it corresponds to a known MITRE ATT&CK technique.
                - Justify the mapping by explaining the intent or effect of the behavior.
                - Extract the correct MITRE ATT&CK technique ID and name.
                - Identify exact evidence strings from either static_analysis, dynamic_analysis, or iocs only.
                - Do not use any evidence from behavior_ttp_mapping for citations.
                - Only include mappings that are confident and valid. Avoid weak or speculative mappings.

                Response only following output format (JSON) and do not add anything else:
                Output Format (JSON array):
                    
                "technique_id": "Txxxx",
                "evidence": [
                      "Exact evidence string 1 from static/dynamic/IoC",
                      "Exact evidence string 2 from static/dynamic/IoC"
                ],
                "reasoning": "Explanation of why this evidence supports the mapping to the technique."
                
                Note:
                – Merge multiple supporting behaviors into a single TTP mapping when appropriate  
                – Avoid duplicate TTP entries  
                – Only use official MITRE ATT&CK techniques
                
                Begin your analysis using the provided inputs:
                static_analysis:
                {processed_static}
                dynamic_analysis:
                {processed_dynamic["signatures"]}

                Response:
                        """
    ans = extract_first_json_array(get_gemini_response(prompt))
    #print(f"Response signatures: \n{get_gemini_response(prompt)}")
    if ans != None:
        for ans_i in ans:
            if ans_i['technique_id'] not in final_result.keys():
                final_result[ans_i['technique_id']] = ans_i['evidence']
            else:
                for e in ans_i['evidence']:
                    final_result[ans_i['technique_id']].append(e)
    return final_result



In [17]:
def calc_rr(rank_list, ground_truth):
    rank = None
    for item in rank_list:
        group_id, score, r = item
        if group_id == ground_truth:
            rank = r
            print(group_id, score, r)
            break
    
    if rank is not None:
        return 1 / rank
    else:
        return 0


In [ ]:
import os
import time
import json

REPORT_DIR = "/kaggle/input/dataset-apt-malware/Winnti/"
with open('/kaggle/input/eval-file/eval_file.json', 'r') as f:
    eval_file = json.load(f)
cnt = 0
CR = 0
cntv2 = 0
for file in os.listdir(REPORT_DIR):
    if cnt == 9: break
    if file in eval_file: continue
    print(file)
    eval_file.append(file)
    with open('/kaggle/working/eval_file.json', 'w') as f:
        json.dump(eval_file, f, indent=4)
    cnt += 1
    report = REPORT_DIR + file
    ground = search_hashv2(file)
    while ground == None:
        ground = search_hashv2(file)
        cntv2 += 1
        if cntv2 == 10:
            break
    cntv2 = 0
    if ground == None:
        cnt -= 1
        continue
    static_path = report + "/" + file + ".json"
    dynamic_path = report + "/" + "report.json"
    start_time = time.time()
    on_process_report_clicked(static_path, dynamic_path)
    final_result = on_generate_clicked()
    if final_result == None: 
        cnt-=1
        end_time = time.time()
        continue
    #print(final_result)
    file_result = "/kaggle/working/" + file + ".json"
    with open(file_result, 'w') as f:
        json.dump(final_result, f, indent=4)
    set_g = set(list(final_result.keys()))
    set_r = set(list(ground.keys())) 
    print(len(set_g & set_r))
    print(len(set_g))
    print(len(set_r))
    CR += len(set_g & set_r) / len(set_r)
    print(f"{file} - CR_sum = {CR:.4f} - CR_cur = {(len(set_g & set_r) / len(set_r)):.4f}")
    print("Reciprocal Rank (RR):", calc_rr(infer_apt(list(final_result.keys())), "G0044"), "Ground Truth:", os.path.basename(os.path.normpath(REPORT_DIR)))
    end_time = time.time()

    elapsed_time = end_time - start_time
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = elapsed_time % 60
    
    print(f"Run time: {hours} h {minutes} m {seconds:.2f} s - {elapsed_time:.2f}s")
